In [ ]:
!pip install gym[atari]

In [1]:
%matplotlib widget

In [2]:
from IPython.core.debugger import set_trace

In [3]:
import torch
from torch import nn
from torch.distributions import Categorical
from torch.optim import Adam, SGD

In [4]:
import numpy

In [5]:
from matplotlib import pyplot as plot

In [6]:
import copy
from time import sleep

In [7]:
import gym

In [9]:
from utils import Buffer, collect_one_episode, normalize_obs, copy_params, avg_params

In [8]:
device='cpu'

In [10]:
class ResLinear(nn.Module):
    def __init__(self, n_in, n_out, act=nn.ReLU()):
        super(ResLinear, self).__init__()
        self.act = act
        self.linear = nn.Linear(n_in, n_out)
        self.bn = nn.BatchNorm1d(n_out)
        
        assert(n_in == n_out)
    
    def forward(self, x):
        h = self.act(self.bn(self.linear(x)))
        return h + x

In [11]:
class Player(nn.Module):
    def __init__(self, n_in=128, n_hid=100, n_out=6):
        super(Player, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_out))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, obs, normalized=False):
        if normalized:
            return self.softmax(self.layers(obs))
        else:
            return self.layers(obs)

In [12]:
class Value(nn.Module):
    def __init__(self, n_in=128, n_hid=100):
        super(Value, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, 1))
    
    def forward(self, obs):
        return self.layers(obs)

In [13]:
env = gym.make('Pong-ram-v0')
# env = gym.make('Assault-ram-v0')

n_frames = 4

In [14]:
# create a policy
player = Player(n_in=128 * n_frames, n_hid=128, n_out=6).to(device)

In [15]:
# create a value estimator
value = Value(n_in=128 * n_frames, n_hid=128).to(device)

In [16]:
# initialize optimizers
opt_player = Adam(player.parameters(), lr=0.0001)
opt_value = Adam(value.parameters(), lr=0.0001)

In [17]:
# initialize replay buffer
replay_buffer = Buffer(max_items=50000, n_frames=n_frames)

In [ ]:
n_iter = 1000
init_collect = 1
n_collect = 1
n_value = 150
n_policy = 150
disp_iter = 1
val_iter = 1

max_len = 1000
batch_size = 1000

ent_coeff = 0. #0.001
discount_factor = .95

value_loss = -numpy.Inf
ret = -numpy.Inf
entropy = -numpy.Inf
valid_ret = -numpy.Inf

return_history = []

for ni in range(n_iter):
    player.eval()

    if numpy.mod(ni, val_iter) == 0:
        _, _, _, _, _, ret_ = collect_one_episode(env, player, max_len=max_len, deterministic=True, n_frames=n_frames)
        return_history.append(ret_)
        if valid_ret == -numpy.Inf:
            valid_ret = ret_
        else:
            valid_ret = 0.9 * valid_ret + 0.1 * ret_
        print('Valid run', ret_, valid_ret)

    # collect some episodes using the current policy
    # and push (obs,a,r,p(a)) tuples to the replay buffer.
    nc = n_collect
    if ni == 0:
        nc = init_collect
    for ci in range(nc):
        o_, r_, c_, a_, ap_, ret_ = collect_one_episode(env, player, max_len=max_len, discount_factor=discount_factor, n_frames=n_frames)
        replay_buffer.add(o_, r_, c_, a_, ap_)
        if ret == -numpy.Inf:
            ret = ret_
        else:
            ret = 0.9 * ret + 0.1 * ret_
    
    player.train()
        
    # fit a value function
    for vi in range(n_value):
        opt_player.zero_grad()
        opt_value.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
#         set_trace()
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_y = torch.from_numpy(numpy.stack([ex['current']['crew'] for ex in batch]).astype('float32')).to(device)
        pred_y = value(batch_x).squeeze()
        loss_ = ((batch_y - pred_y) ** 2)
        
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        batch_pi = player(batch_x, normalized=True)
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_value.step()
        
    if value_loss < 0.:
        value_loss = loss_.mean().item()
    else:
        value_loss = 0.9 * value_loss + 0.1 * loss_.mean().item()
    
    if numpy.mod(ni, disp_iter) == 0:
        print('# plays', (ni+1) * n_collect, 'return', ret, 'value_loss', value_loss, 'entropy', -entropy)
    
    # fit a policy
    for pi in range(n_policy):
        opt_player.zero_grad()
        opt_value.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_r = torch.from_numpy(numpy.stack([ex['current']['crew'] for ex in batch]).astype('float32')[:,None]).to(device)
        batch_v = value(batch_x)
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)

        batch_pi = player(batch_x, normalized=True)
        
        logp = torch.log(batch_pi.gather(1, batch_a.long()))
        
        # advantage
        adv = batch_r - batch_v.clone().detach()
        
        loss = -(adv * logp)
        
        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss
        
        # entropy regularization: though, it doesn't look necessary in this specific case.
        ent = (batch_pi * torch.log(batch_pi)).sum(1)
        
        if entropy == -numpy.Inf:
            entropy = ent.mean().item()
        else:
            entropy = 0.9 * entropy + 0.1 * ent.mean().item()
        
        loss = (loss + ent_coeff * ent).mean()
        
        loss.backward()
        opt_player.step()

Valid run -20.0 -20.0
# plays 1 return -15.0 value_loss 0.042173080146312714 entropy inf
Valid run -16.0 -19.6
# plays 2 return -14.4 value_loss 0.040148214995861055 entropy 1.5901245361764969
Valid run -17.0 -19.34
# plays 3 return -14.96 value_loss 0.038919142670929434 entropy 1.4911581890997518
Valid run -16.0 -19.006
# plays 4 return -15.364 value_loss 0.037619841951876884 entropy 1.4017773590347036
Valid run -17.0 -18.8054
# plays 5 return -15.627600000000001 value_loss 0.036553559586778285 entropy 1.3694009865218266
Valid run -19.0 -18.824859999999997
# plays 6 return -15.864840000000003 value_loss 0.03503792022351176 entropy 1.3111446587394813
Valid run -16.0 -18.542374
# plays 7 return -15.778356000000002 value_loss 0.03422884823099152 entropy 1.2405346815812452
Valid run -17.0 -18.3881366
# plays 8 return -15.900520400000001 value_loss 0.03305805692418255 entropy 1.1998670027407692
Valid run -19.0 -18.44932294
# plays 9 return -16.110468360000002 value_loss 0.03217406282732342

In [22]:
plot.figure()

plot.plot(return_history)
plot.grid(True)
plot.xlabel('# of plays x {}'.format(n_collect))
plot.ylabel('Return over the episode of length {}'.format(max_len))

plot.show()
plot.savefig('return_log.pdf', dpi=150)

FigureCanvasNbAgg()

In [21]:
next(player.parameters()).device

device(type='cpu')

In [ ]:
# let the final policy play the pong longer
player.eval()
_, _, _, _, _, ret_ = collect_one_episode(env, player, max_len=1000000, deterministic=True, rendering=True)

RecursionError: maximum recursion depth exceeded